![logo](../picture/license_header_logo.png)

> **Copyright (c) 2020-2021 CertifAI Sdn. Bhd.**<br>
 <br>
This program is part of OSRFramework. You can redistribute it and/or modify
<br>it under the terms of the GNU Affero General Public License as published by
<br>the Free Software Foundation, either version 3 of the License, or
<br>(at your option) any later version.
<br>
<br>This program is distributed in the hope that it will be useful,
<br>but WITHOUT ANY WARRANTY; without even the implied warranty of
<br>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
<br>GNU Affero General Public License for more details.
<br>
<br>You should have received a copy of the GNU Affero General Public License
<br>along with this program.  If not, see <http://www.gnu.org/licenses/>.
<br>

Authored by: [BK Yeoh](boonkhai.yeoh@certifai.ai)

# Model Serialization & Model Conversion
## Introduction
This tutorial is to demonstrate the ways of model serialization and model conversion. It consists of 2 major parts which are:
1. Demonstration of Model Serialization
2. Demonstration of Model Conversion

## What is serialization?
It is a saving procedure that uses to represent an object with a stream of bytes. The goal is to save the model's parameters and coefficients to a file so that the model training and parameter optimization steps do not have to be repeated on new data to apply during the production.

>The most common method is to serialize the model using some particular format after training and deserialize that model in the production environment.

## What is model conversion? 
Model conversion is a promising technology for improving framework interoperability that transforms a source model into  another target framework format.

## Notebook Outline
Below is the outline for this tutorial:
* [Helper Function](#Helper)
* [Load Model](#load)
* [Image transform](#transform)
* [Inference Function](#Inference)
* [Load Model State Dict](#Load)
* [Ways of (de)serialize model](#Ways)
    * [Save Model using Python Pickle](#Pickle)
    * [Save Model using Joblib](#Joblib)
    * [Save model using TorchScript](#TorchScript)
* [Model Conversion](#Conversion)
    * [Save Model in ONNX format](#ONNXformat)
    * [Save Model in Protocol Buffers](#ProtocolBuffers)
    * [Save model in Json format](#Jsonformat)
* [Tips for Saving Your Model](#Tips)
* [Summary](#Summary)
* [Reference](#Reference)

## What will we accomplish?
1. Understand different ways of model serialization. 
2. Understand the process of model conversion and usage of each file format.

First, let's import the package needed.

In [1]:
import os
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.onnx
import torchvision
from torchvision import transforms,datasets
from torch.utils.data import DataLoader
from time import time
import warnings
import utils 

## Helper Function <a id=Helper></a>

In [2]:
# Generate folder to store the model
gen_model_path = 'generated_model/demo'
dir_list = ['pickle','joblib','torchscript','onnx','protocol_buffers','json'] 
utils.folder_generator(gen_model_path,dir_list)

Directory generated_model/demo already exists, skipping create
Subfolder pickle already exists, skipping create
Subfolder joblib already exists, skipping create
Subfolder torchscript already exists, skipping create
Subfolder onnx already exists, skipping create
Subfolder protocol_buffers already exists, skipping create
Subfolder json already exists, skipping create


In [3]:
# Download the data from wasabisys         
source = 'https://s3.eu-central-1.wasabisys.com/certifai/deployment-training-labs/fruits_image_classification-20210604T123547Z-001.zip'
target = '../resources/data/'
filename = 'fruits_image_classification.zip'
utils.download(source, target, filename,zip_file = True)

file already exists, skipping download


In [4]:
# Download the model from wasabisys 
model_url = "https://s3.eu-central-1.wasabisys.com/certifai/deployment-training-labs/models/fruit_classifier_state_dict.pt"
modelname = "fruit_classifier_state_dict.pt"
utils.download(model_url, gen_model_path, modelname)

file already exists, skipping download


First, let get the dirty test dataset folder path that you have downloaded in the previous lesson.

In [5]:
# The data is located in the data folder
datadir = Path().resolve().parent/'resources/data/'
dirtytestdir = datadir/'fruits_image_classification/dirty_test/'

## Load Model <a id=load></a>
Download the serialized model in the previous lesson. 

Define Model Architecture

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.relu2 = nn.ReLU()
        self.fc1 = nn.Linear(18496, 120) # Note that the input of this layers is depending on your input image sizes 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 3) 
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu1(self.conv1(x)))
        x = self.pool(self.relu2(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Image transform<a id=transform></a>

In [7]:
# We will need our input in tensors form, must `transforms.ToTensor(),`
val_transform = transforms.Compose([
        transforms.Resize(150), # You can set to higher resolution for better result
        transforms.CenterCrop(150),
        transforms.ToTensor(),
    ])

## Inference Function<a id=Inference></a>

In [8]:
def inference(best_model, testdir = dirtytestdir , image_transforms = val_transform):
    best_model.eval()
    test_data = datasets.ImageFolder(root = testdir, transform = image_transforms)
    testloader = DataLoader(test_data, len(test_data), shuffle=False)
    for images, labels in testloader:
        correct = 0
        y_pred = best_model(images)
        predictions = torch.max(y_pred, 1)[1]
        correct += (predictions == labels).sum().item()
        accuracy = correct / len(test_data) 
    print(f"Test Accuracy: {accuracy}")

## Load Model State Dict<a id=Load></a>
### What is a state_dict?
* It is a Python dictionary object that maps each layer to its parameter tensor.
* When saving the model for inference, saving the model’s state_dict will give the flexibility for restoring the model later.

In [9]:
# To load a saved state dict, we need to instantiate the model first
model = Net()

# Notice that the load_state_dict() function takes a dictionary object, NOT a path to a saved object. 
# This means that you must deserialize the saved state_dict before you pass it to the load_state_dict() function.
model.load_state_dict(torch.load(os.path.join(gen_model_path,'fruit_classifier_state_dict.pt')))
model

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (fc1): Linear(in_features=18496, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=3, bias=True)
  (relu): ReLU()
)

## Ways of (de)serialize model<a id=Ways></a>

### Save Model using Python Pickle<a id=Pickle></a>
#### What is Python Pickle
* It’s a standard Python tool to saving object (serialization) into a bytes file
* The object can be a model or a data

#### Advantages of using Python Pickle 
* Quicksave and restore learning model

#### The downside of Python Pickle 
* Does not have security


In [10]:
import pickle

In [11]:
# save the model to disk
pickle_wt = time()
pickle_filename = os.path.join(gen_model_path,'pickle/fruit_classifier.pt')
pickle.dump(model, open(pickle_filename, 'wb'))
print("Time for saving model with pickle =>", time()-pickle_wt)

Time for saving model with pickle => 0.00899815559387207


In [12]:
# load the model from disk
pickle_rt = time()
loaded_model_pk = pickle.load(open(pickle_filename, 'rb'))
print("Time for loading file size with pickle", os.path.getsize(pickle_filename),"KB =>", time()-pickle_rt)
inference(loaded_model_pk)

Time for loading file size with pickle 8936493 KB => 0.013002157211303711
Test Accuracy: 0.7222222222222222


### Save Model using Joblib<a id=Joblib></a>
#### What is Joblib
Joblib is part of the SciPy ecosystem and provides utilities for pipelining Python jobs.
It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently.
This can be useful for some machine learning algorithms that require a lot of parameters or store the entire dataset

#### Advantages of using Joblib 
* The Joblib library is intended to be a replacement for Pickle by offers a bit simpler workflow compared to Pickle.
* It works with both file objects and string filenames.
* If the model contains large data arrays, each array will be stored in its own file, but the save and restore procedure will remain the same.
* Joblib also supports various compression methods, such as 'zlib,' 'gzip,' and 'bz2', as well as different levels of compression.


In [13]:
import joblib

In [14]:
# save the model to disk
joblib_wt = time()
joblib_filename = os.path.join(gen_model_path,'joblib/fruit_classifier.pt')
joblib.dump(model, joblib_filename)
print("Time for saving model with joblib =>", time()-joblib_wt)

Time for saving model with joblib => 0.009999990463256836


In [15]:
# load the model from disk
joblib_rt = time()
loaded_model_jb = joblib.load(joblib_filename)
print("Time for loading file size with joblib", os.path.getsize(joblib_filename),"KB =>", time()-joblib_rt)
inference(loaded_model_jb)

Time for loading file size with joblib 8936493 KB => 0.014000654220581055
Test Accuracy: 0.7222222222222222


### Biggest Drawback of Pickle and Joblib
#### Python version compatibility 
* Both tools are not recommended to (de)serialize objects across different Python versions, though it may work with minor version changes.

#### Model compatibility
* The internal structure of the model must remain constant between saving and reload for both tools.

#### Security
* Both tools may contain malicious code, so restoring data from untrusted or unauthenticated sources is not recommended.


### Save model using TorchScript <a id=TorchScript></a>

#### PyTorch Ecosystem
PyTorch has two distinct modes for dealing with the research and production environments.<br>

**Eager mode** - It is designed to facilitate faster prototyping, training, and experimentation.<br>
**Script mode** - It is geared toward the production use case. It is made up of two parts: `PyTorch JIT` and `TorchScript`.<br>

<div><img  src="../picture/eager_to_script.png",width=850,height=200></div>
<center><b>Tools to transition from eager to script</b></center>

#### What is script mode?
Script mode creates an intermediate representation (IR) from the PyTorch Eager module. The IR is internally optimized and utilizes PyTorch JIT compilation at runtime.
<div><img  src="../picture/script_mode.png",width=850,height=200></div>
<center><b>Details of script mode</b></center>

#### What is PyTorch JIT
It is a compiler and language infrastructure for machine learning and an optimized compiler for PyTorch programs.
1. It is a lightweight threadsafe interpreter
2. Supports easy to write custom transformations
3. It’s not just for inference as it has auto diff support

#### What is TorchScript?
TorchScript is an intermediate representation of a PyTorch model (subclass of nn.Module) that can then be run in a high-performance environment such as C++.

>Torch Script is a way to create serializable and optimizable models from PyTorch code. Any code written in Torch Script can be saved from your Python process and loaded in a process where there is no Python dependency. - *Pytorch.org*

TorchScript is a static high-performance subset of Python language, specialized for ML applications. It supports
1. Complex control flows
2. Common data structures
3. User-defined classes

#### Why Script mode?
1. **Portability**<br>
Portability enables models to be deployed in multithreaded inference servers, mobile devices, and automobiles which difficult due to the tight coupling of models to the Python runtime. 

2. **Performance**<br>
Optimize common patterns in neural networks to improve inference latency. There is numerous further optimization that cannot achieve with the level of dynamism in the Python language.

#### Save model using `JIT Trace`
`torch.jit.trace` take a data instance and your trained eager module as input. The tracer runs the supplied module and records the tensor operations performed. This recording is turned into a TorchScript module.

Drawbacks
it omits: 
- control flow,
- data structures
- python constructs
- create unfaithful representations without any warnings

In [16]:
# Build an input example (batch size, channels, image height, image width) which image size is set in image transform 
x = torch.randn(10, 3, 150, 150, requires_grad=True)
traced_model = torch.jit.trace(model,x)
print(traced_model)

Net(
  original_name=Net
  (conv1): Conv2d(original_name=Conv2d)
  (relu1): ReLU(original_name=ReLU)
  (pool): MaxPool2d(original_name=MaxPool2d)
  (conv2): Conv2d(original_name=Conv2d)
  (relu2): ReLU(original_name=ReLU)
  (fc1): Linear(original_name=Linear)
  (fc2): Linear(original_name=Linear)
  (fc3): Linear(original_name=Linear)
  (relu): ReLU(original_name=ReLU)
)


TorchScript records its definitions in an Intermediate Representation (or IR), commonly referred to in Deep learning as a graph. We can examine the graph with the `.graph` property:


In [17]:
# Intermediate representation (IR) of the model
print(traced_model.graph)

graph(%self.1 : __torch__.Net,
      %input.1 : Float(10, 3, 150, 150, strides=[67500, 22500, 150, 1], requires_grad=1, device=cpu)):
  %155 : __torch__.torch.nn.modules.linear.___torch_mangle_3.Linear = prim::GetAttr[name="fc3"](%self.1)
  %152 : __torch__.torch.nn.modules.linear.___torch_mangle_2.Linear = prim::GetAttr[name="fc2"](%self.1)
  %149 : __torch__.torch.nn.modules.activation.___torch_mangle_4.ReLU = prim::GetAttr[name="relu"](%self.1)
  %148 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="fc1"](%self.1)
  %145 : __torch__.torch.nn.modules.activation.___torch_mangle_1.ReLU = prim::GetAttr[name="relu2"](%self.1)
  %144 : __torch__.torch.nn.modules.conv.___torch_mangle_0.Conv2d = prim::GetAttr[name="conv2"](%self.1)
  %141 : __torch__.torch.nn.modules.pooling.MaxPool2d = prim::GetAttr[name="pool"](%self.1)
  %140 : __torch__.torch.nn.modules.activation.ReLU = prim::GetAttr[name="relu1"](%self.1)
  %139 : __torch__.torch.nn.modules.conv.Conv2d = prim::GetAttr[

However, this is a very low-level representation and most of the information contained in the graph is not useful for end users. Instead, we can use the `.code` property to give a Python-syntax interpretation of the code:

In [18]:
print(traced_model.code)

def forward(self,
    input: Tensor) -> Tensor:
  _0 = self.fc3
  _1 = self.fc2
  _2 = self.relu
  _3 = self.fc1
  _4 = self.relu2
  _5 = self.conv2
  _6 = self.pool
  _7 = (self.relu1).forward((self.conv1).forward(input, ), )
  _8 = (_4).forward((_5).forward((_6).forward(_7, ), ), )
  input0 = torch.flatten((_6).forward1(_8, ), 1, -1)
  _9 = (_2).forward((_3).forward(input0, ), )
  _10 = (_0).forward((_2).forward1((_1).forward(_9, ), ), )
  return _10



##### (De)serialize `traced_model`

In [19]:
# Serialize model to disk
torch_path = os.path.join(gen_model_path,"torchscript/")
torch.jit.save(traced_model,torch_path +"traced_model.pt")

In [20]:
# Deserialize model from disk
loaded_traced_model = torch.jit.load(torch_path +"traced_model.pt")
inference(loaded_traced_model)

Test Accuracy: 0.7222222222222222


#### Save model using `JIT Script`
Unfortunately, things like control flow are erased by using `torch.jit.trace`. 

In [21]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

In [22]:
new_model = MyDecisionGate()

In [23]:
warnings.filterwarnings("ignore")
traced_new_model = torch.jit.trace(new_model,x)
print(traced_new_model.code)

def forward(self,
    x: Tensor) -> Tensor:
  return torch.neg(x)



How can we faithfully represent this module in TorchScript? We provide a script compiler, which does direct analysis of your Python source code to transform it into TorchScript. 

In [24]:
scripted_gate = torch.jit.script(new_model)
print(scripted_gate.code)

def forward(self,
    x: Tensor) -> Tensor:
  _0 = bool(torch.gt(torch.sum(x, dtype=None), 0))
  if _0:
    _1 = x
  else:
    _1 = torch.neg(x)
  return _1



##### (De)serialize `script_model`

In [25]:
script_model = torch.jit.script(model)

In [26]:
torch_path = os.path.join(gen_model_path,"torchscript/")
torch.jit.save(script_model,torch_path +"script_model.pt")

In [27]:
loaded_script_model = torch.jit.load(torch_path +"script_model.pt")
inference(loaded_script_model)

Test Accuracy: 0.7222222222222222


### Advantages of using TorchScript
* TorchScript decouples your model from any runtime environment, improving inference time. It eliminates Python's GIL, which is a significant bottleneck for multithreaded inference.
* Focusing on optimising entire programmes.
* It optimises common neural network patterns automatically to improve latency and throughput.
TorchScript modules can be exported to a wide range of environments, from C++ servers to mobile devices.


# Model Conversion<a id=Conversion></a>
We can convert the model from the PyTorch framework to others such as TensorFlow and Keras. These can achieve by using the model converters from the official website or third-party package.

Example of the third-party package converter:

converter|	mxnet|	caffe| 	caffe2|	CNTK|	theano/lasagne|	neon|	pytorch|	torch|	keras| 	darknet|	tensorflow| 	chainer|	coreML/iOS|
---|	---|	---| 	---|	---|	---|	---|	---|	---|	---| 	---|	---| 	---|	---
pytorch| 	[MMdnn](https://github.com/Microsoft/MMdnn)|	[MMdnn](https://github.com/Microsoft/MMdnn) [pytorch2caffe](https://github.com/longcw/pytorch2caffe) [pytorch-caffe-darknet-convert](https://github.com/nanhui69/pytorch-caffe-darknet-convert)|	Part of Pytorch now|	ONNX [MMdnn](https://github.com/Microsoft/MMdnn)|	None|	None|	-|	None|	[MMdnn]((https://github.com/Microsoft/MMdnn)) [pytorch2keras](https://github.com/gmalivenko/pytorch2keras) [onnx2keras](https://github.com/waltYeh/onnx2keras)|	None| [onnx-tf](https://github.com/onnx/onnx-tensorflow)|	None|	[onnx-coreml](https://github.com/onnx/onnx-coreml)

## Save Model in ONNX format<a id=ONNXformat></a>
PyTorch also supports saving model as ONNX (Open Neural Network Exchange) file type, which is an open format built to represent machine learning models. ONNX provides a standard that would allow portability and interoperability between the already well-known Deep Learning frameworks like PyTorch, Tensorflow, Caffe2, MXNet, etc. For example, we can train the model in one framework and export the model in ONNX format to perform inference in another framework.
Also, we can utilise the ONNX Runtime to accelerate inference and allow the model to run in different hardware architectures..

In [28]:
# Build an input example (batch size, channels, image height, image width) which image size is set in image transform 
# As the argument in torch.onnx.export
x = torch.randn(10, 3, 150, 150, requires_grad=True)

In [29]:
# Export the model
onnx_filepath = os.path.join(gen_model_path,'onnx/fruit_classifier.onnx')
torch.onnx.export(model, x,onnx_filepath, 
                  verbose = True, input_names = ['input'], output_names = ['output'])

graph(%input : Float(10, 3, 150, 150, strides=[67500, 22500, 150, 1], requires_grad=1, device=cpu),
      %conv1.weight : Float(6, 3, 5, 5, strides=[75, 25, 5, 1], requires_grad=1, device=cpu),
      %conv1.bias : Float(6, strides=[1], requires_grad=1, device=cpu),
      %conv2.weight : Float(16, 6, 5, 5, strides=[150, 25, 5, 1], requires_grad=1, device=cpu),
      %conv2.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %fc1.weight : Float(120, 18496, strides=[18496, 1], requires_grad=1, device=cpu),
      %fc1.bias : Float(120, strides=[1], requires_grad=1, device=cpu),
      %fc2.weight : Float(84, 120, strides=[120, 1], requires_grad=1, device=cpu),
      %fc2.bias : Float(84, strides=[1], requires_grad=1, device=cpu),
      %fc3.weight : Float(3, 84, strides=[84, 1], requires_grad=1, device=cpu),
      %fc3.bias : Float(3, strides=[1], requires_grad=1, device=cpu)):
  %11 : Float(10, 6, 146, 146, strides=[127896, 21316, 146, 1], requires_grad=1, device=cpu) = onnx:

## Save Model in Protocol Buffers<a id=ProtocolBuffers></a>
To save the model into protocol buffers `.pb` format. You are required to convert the model from ONNX to TensorFlow by using `onnx_tf`
#### Procedure for  `onnx_tf ` installation
Install `onnx_tf`  master branch from source.
1. Install `TensorFlow >= 2.4.1` and `tensorflow-addons==0.13.0` using the following commands.

    `pip install tensorflow==2.5.0` <br>
    `pip install tensorflow-addons==0.13.0` <br>

2. Run git clone https://github.com/onnx/onnx-tensorflow.git && `cd onnx-tensorflow`.
3. Run `pip install -e .`.

In [30]:
from onnx_tf.backend import prepare
import onnx

In [31]:
# Load ONNX model format and convert to tensorflow model format 
onnx_model = onnx.load(onnx_filepath)
tf_rep = prepare(onnx_model)

In [32]:
# Covert the tensorflow format to Protocol Buffers format
warnings.filterwarnings("ignore")
pb_filepath = os.path.join(gen_model_path,"protocol_buffers")
tf_rep.export_graph(pb_filepath)

INFO:tensorflow:Assets written to: generated_model/demo\protocol_buffers\assets


INFO:tensorflow:Assets written to: generated_model/demo\protocol_buffers\assets


Run the below code to validate the Protocol Buffers format The input shape should be the same as the input when serializing the model in ONNX format which is `[10,3,150,150]` and the output shape should be the expected output shape base on the model's configuration which is `[10,3]`
![image](https://user-images.githubusercontent.com/59526258/121812655-3f5d2c00-cc9b-11eb-9062-b4b9c73ce91b.png)

In [33]:
!saved_model_cli show --dir {pb_filepath} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input'] tensor_info:
        dtype: DT_FLOAT
        shape: (10, 3, 150, 150)
        name: serving_default_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output'] tensor_info:
        dtype: DT_FLOAT
        shape: (10, 3)
        name: PartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
        Named Argument #1
          input

  Function Name: 'gen_tensor_dict'


2021-08-09 13:09:23.669529: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-08-09 13:09:23.669558: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Save model in Json format<a id=Jsonformat></a>
To save the model into JSON `.json` format. You are required to convert the model from ONNX to Keras by using `onnx_to_keras`

In [34]:
from onnx2keras import onnx_to_keras
k_model = onnx_to_keras(onnx_model, ['input'])
k_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 3, 150, 150)]     0         
_________________________________________________________________
11 (Conv2D)                  (None, 6, 146, 146)       456       
_________________________________________________________________
12 (Activation)              (None, 6, 146, 146)       0         
_________________________________________________________________
13 (MaxPooling2D)            (None, 6, 73, 73)         0         
_________________________________________________________________
14 (Conv2D)                  (None, 16, 69, 69)        2416      
_________________________________________________________________
15 (Activation)              (None, 16, 69, 69)        0         
_________________________________________________________________
16 (MaxPooling2D)            (None, 16, 34, 34)        0     

In [35]:
# Serialize model to JSON
import json
json_model_path = os.path.join(gen_model_path,'json')
file_name = "fruit_classifier.json"
complete_name = os.path.join(json_model_path,file_name)
k_model_json = k_model.to_json()

with open(complete_name, "w") as json_file:
    json_file.write(k_model_json)   

In [36]:
# Load the json file to validate the conversion
json.load(open(complete_name))

{'class_name': 'Functional',
 'config': {'name': 'model',
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': [None, 3, 150, 150],
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'input'},
    'name': 'input',
    'inbound_nodes': []},
   {'class_name': 'Conv2D',
    'config': {'name': '11',
     'trainable': True,
     'dtype': 'float32',
     'filters': 6,
     'kernel_size': [5, 5],
     'strides': [1, 1],
     'padding': 'valid',
     'data_format': 'channels_first',
     'dilation_rate': [1, 1],
     'groups': 1,
     'activation': 'linear',
     'use_bias': True,
     'kernel_initializer': {'class_name': 'Zeros', 'config': {}},
     'bias_initializer': {'class_name': 'Zeros', 'config': {}},
     'kernel_regularizer': None,
     'bias_regularizer': None,
     'activity_regularizer': None,
     'kernel_constraint': None,
     'bias_constraint': None},
    'name': '11',
    'inbound_nodes': [[['input', 0, 0, {}]]]},
   {'cl

In [37]:
from keras.models import model_from_json

# load json and create model
json_file = open(complete_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 3, 150, 150)]     0         
_________________________________________________________________
11 (Conv2D)                  (None, 6, 146, 146)       456       
_________________________________________________________________
12 (Activation)              (None, 6, 146, 146)       0         
_________________________________________________________________
13 (MaxPooling2D)            (None, 6, 73, 73)         0         
_________________________________________________________________
14 (Conv2D)                  (None, 16, 69, 69)        2416      
_________________________________________________________________
15 (Activation)              (None, 16, 69, 69)        0         
_________________________________________________________________
16 (MaxPooling2D)            (None, 16, 34, 34)        0     

## Tips for Saving Your Model<a id=Tips></a>
This section discusses some important factors to consider when finishing your machine learning models according to Jason Brownlee.

* **Python version**: Keep an eye out for the Python version. When load and deserialize the model, the same version of Python is needed that was used to serialise it.
* **Library Editions**: When deserializing a saved model, the version of all major libraries used in machine learning project almost certainly needs to be the same.

# Summary<a id=Summary></a>
From this tutorial, you should have learned:
1. Different methods of (de)serialize model.
2. Model Conversion to different types of model format. 

Congratulations, that concludes this lesson.

# Reference<a id=Reference></a>
1. [Machine Learning Mastery With Python - Jason Brownlee ](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)
2. [Deep Learning Model Convertors](https://blog.csdn.net/baidu_40840693/article/details/86288667?ops_request_misc=&request_id=&biz_id=102&utm_term=pytorch%20model%20conversion&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-2-.first_rank_v2_pc_rank_v29&spm=1018.2226.3001.4187)
3. [How to use gRPC API to Serve a Deep Learning Model?](https://towardsdatascience.com/serving-deep-learning-model-in-production-using-fast-and-efficient-grpc-6dfe94bf9234)
4. [onnx2keras](https://github.com/waltYeh/onnx2keras)